In [1]:
import os
import glob

import torch
import torch.nn as nn
import numpy as np
from models.eeg_AE import eeg_encoder as Encoder

from models.eeg_AE import eeg_decoder as Decoder

/home/user/anaconda3/envs/SD3/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
number        = 50000
original_name = f"EEG_condition_{number}.pth"
output_name   = f"EEGEncoder_BLIP2_sub4_{number}"
original_path = f"./ckpt_dir/new_version/EEG_blip2_77_1024_sub4"
original_path = os.path.join(original_path, original_name)

In [3]:
original_path

'./ckpt_dir/new_version/EEG_blip2_77_1024_sub4/EEG_condition_50000.pth'

In [4]:
test = Encoder( in_seq = 440, 
        in_channels = 128, 
        out_seq = 1024,
        out_channels=77,
        dims = [64, 128, 256, 512, 1024], 
        shortcut = 1, 
        dropout = 0.5, 
        groups = 32,
        layer_mode = "conv",  
        block_mode = 'res', 
        down_mode = 'linear',
        pos_mode = 'trunc', 
        n_layer =2, 
        n_head = 64,
        dff_factor= 2,
        stride = 4, 
        skip_mode= 'down',
        global_attn = True).cuda()
dict_model = torch.load(original_path)
m, u =test.load_state_dict(dict_model['net'], strict=False)

In [5]:
for n,v in dict_model['net'].items():
    print(n)
    if "Encoder" in  n:
        print(n[8:])

Encoder.posEmbed.0
posEmbed.0
Encoder.posEmbed.1
posEmbed.1
Encoder.posEmbed.2
posEmbed.2
Encoder.posEmbed.3
posEmbed.3
Encoder.posEmbed.4
posEmbed.4
Encoder.in_layer.conv.weight
in_layer.conv.weight
Encoder.in_layer.conv.bias
in_layer.conv.bias
Encoder.out_layer.conv.weight
out_layer.conv.weight
Encoder.out_layer.conv.bias
out_layer.conv.bias
Encoder.skips.0.conv.weight
skips.0.conv.weight
Encoder.skips.1.conv.weight
skips.1.conv.weight
Encoder.skips.2.conv.weight
skips.2.conv.weight
Encoder.skips.3.conv.weight
skips.3.conv.weight
Encoder.skips.4.conv.weight
skips.4.conv.weight
Encoder.skips_agg.0.linear.weight
skips_agg.0.linear.weight
Encoder.skips_agg.2.linear.weight
skips_agg.2.linear.weight
Encoder.skips_agg.4.linear.weight
skips_agg.4.linear.weight
Encoder.skips_agg.5.mha.wq.conv.weight
skips_agg.5.mha.wq.conv.weight
Encoder.skips_agg.5.mha.wk.conv.weight
skips_agg.5.mha.wk.conv.weight
Encoder.skips_agg.5.mha.wv.conv.weight
skips_agg.5.mha.wv.conv.weight
Encoder.skips_agg.5.mha.

In [6]:
encoder_state_dict = {n[8:] : v for n,v in dict_model['net'].items() if "Encoder" in n }
encoder_state_dict

{'posEmbed.0': tensor([[[ 3.2329e-02,  4.5969e-02, -6.3933e-03,  ...,  2.0737e-02,
           -5.0797e-02,  1.1792e-02],
          [-7.3706e-02, -2.2020e-02, -3.5574e-02,  ..., -4.0211e-02,
            2.3347e-03, -4.2416e-02],
          [ 7.1985e-06, -4.4901e-02, -2.9137e-02,  ..., -5.7455e-02,
            3.8557e-02,  3.1388e-03],
          ...,
          [-3.7835e-02, -5.5382e-03,  1.0517e-02,  ...,  6.1049e-02,
           -3.3775e-02,  4.5525e-02],
          [ 1.4640e-02,  3.3772e-02,  9.1896e-03,  ..., -4.3430e-02,
            1.4927e-02, -1.3390e-02],
          [-2.4838e-02,  2.0952e-02,  2.2139e-02,  ..., -2.7147e-03,
            3.9859e-03,  3.7018e-02]]], device='cuda:0'),
 'posEmbed.1': tensor([[[-0.0371, -0.0130,  0.0326,  ...,  0.0548,  0.0047, -0.0318],
          [-0.0186, -0.0234,  0.0175,  ..., -0.0433,  0.0217,  0.0471],
          [ 0.0285,  0.0285, -0.0032,  ...,  0.0413, -0.0113, -0.0474],
          ...,
          [-0.0150,  0.0136,  0.0352,  ...,  0.0084,  0.0063, -0

In [7]:
m, u =test.load_state_dict(encoder_state_dict, strict=False)

In [8]:
torch.save({"net": encoder_state_dict,},
                os.path.join("pretrain_models", f"{output_name}.pth"))